In [1]:
import pandas as pd
from pandas.io.common import EmptyDataError
from tqdm.autonotebook import trange, tqdm

from plotly.subplots import make_subplots
import plotly.graph_objects as go
import datetime
from datetime import datetime, timedelta, timezone
from pytz import timezone
import humanize

import os
import re

import dash
import dash_core_components as dcc
import dash_bootstrap_components as dbc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import time
import numpy as np
import math
import seaborn as sns

import requests
import io
import warnings

from collections import OrderedDict

In [2]:
def import_df(filename, index = False):
    try:
        if index == False:
            df = pd.read_csv(filename)
        else:
            df = pd.read_csv(filename, index_col=0)
    except EmptyDataError:
        df = pd.DataFrame()
    return df

def set_date(date_str, old_tz, dt_format = "%d/%m/%Y %H:%M:%S"):
    if date_str == 'NaT':
        return pd.NaT
    else:
        datetime_set_naive = datetime.strptime(date_str, dt_format)
        datetime_set_old = timezone(old_tz).localize(datetime_set_naive)
        datetime_set_utc = datetime_set_old.astimezone(timezone('UTC'))
        return datetime_set_utc

In [3]:
info = OrderedDict()
info['setup'] = import_df("Temp/Data/info/info_setup.csv", index = True)
date_end = set_date(info['setup'].loc['date_end_utc','value'].replace('+00:00',''), "UTC", "%Y-%m-%d %H:%M:%S.%f")
diff = datetime.now(timezone('UTC')) - date_end
last_date = humanize.naturaldelta(diff)

In [6]:
if diff > timedelta(hours = 4):
    from Scripts import GetLatestData

5/371 [00:12<00:38,  7.16it/s]
Open files to import:   0%|          | 0/4 [00:00<?, ?it/s]Urban Observatory data up-to-date!

Creating plot dictionaries: 100%|██████████| 5/5 [00:00<00:00,  9.26it/s]


In [7]:
#Import data from temp_files
info = OrderedDict()
chart_dfs_mlt = []
plot_pars = []

data_folder = "Temp/Data/"

for folder in tqdm(os.listdir(data_folder), desc = "Open files to import"):
    if folder.startswith('info'):
        for filename in os.listdir(data_folder + "info"):
            file_id = filename.replace('info_', '')
            file_id = file_id.replace('.csv', '')
            info[file_id] = import_df(data_folder + "info/" + filename, index = True)
    if folder.startswith('chart_dfs_mlt'):
        for filename in os.listdir(data_folder + "chart_dfs_mlt"):
            chart_dfs_mlt.append(import_df(data_folder + "chart_dfs_mlt/" + filename))
    if folder.startswith('plot_pars'):
        for filename in os.listdir(data_folder + "plot_pars"):
            plot_pars.append(import_df(data_folder + "plot_pars/" + filename, index = True))

charts = info['charts'].index.to_list()
date_end = set_date(info['setup'].loc['date_end_utc','value'].replace('+00:00',''), "UTC", "%Y-%m-%d %H:%M:%S.%f")

Open files to import: 100%|██████████| 3/3 [00:03<00:00,  1.33s/it]


In [121]:
chart_figs = []
for chart in charts:
    if info['charts'].loc[chart, 'chart_status'] == 'ON':
        chart_figs.append({})
        selected_plot_info = info['plots'].loc[info['plots'].index == chart][info['plots'].loc[info['plots'].index == chart]['plot'].isin(chart_dfs_mlt[chart].Plot.unique().tolist())]
        # For each plot
        for plot in tqdm(selected_plot_info['plot'].to_list(), desc = "Creating plots for chart " + str(chart)):
            
            fig_par = info['plots'].loc[info['plots'].index == chart].query('plot == "' + plot + '"')
            fig = go.Figure()
            
            #Add traces
            for i in range(0, len(chart_dfs_mlt[chart].query('Plot == "' + plot + '"').Parameter.unique())):
                par = chart_dfs_mlt[chart].query('Plot == "' + plot + '"').Parameter.unique()[i]
                plot_par = plot_pars[chart].query('parameter == "' + par + '"')
                if len(plot_par) != 0:
                    x_data = chart_dfs_mlt[chart][chart_dfs_mlt[chart].Parameter == par].DateTime
                    y_data = chart_dfs_mlt[chart][chart_dfs_mlt[chart].Parameter == par].Value
                    y_error = chart_dfs_mlt[chart][chart_dfs_mlt[chart].Parameter == par].Error

                    if plot_par['line'].values == True:
                        if plot_par['show_in_legend'].values == True:
                            if plot_par['point'].values == False and plot_par['bar'].values == False:
                                legend_show = True
                            else:
                                legend_show = False
                        else:
                            legend_show = False
                        fig.add_trace(go.Scatter(x=x_data, y=y_data, mode='lines',
                                                name=plot_par['parameter_lab'][0], 
                                                line=dict(color=plot_par['colour'][0], width=2),
                                                connectgaps=True,
                                                legendgroup=plot_par['parameter_lab'][0],
                                                showlegend=legend_show
                                                ))
                    if plot_par['point'].values == True:
                        fig.add_trace(go.Scatter(x=x_data, y=y_data, mode='markers',
                                                name=plot_par['parameter_lab'][0], 
                                                marker=dict(color=plot_par['fill'][0],
                                                line=dict(color=plot_par['colour'][0],width=1)),
                                                connectgaps=True,
                                                legendgroup=plot_par['parameter_lab'][0],
                                                showlegend=bool(plot_par['show_in_legend'][0]),
                                                error_y=dict(
                                                    type='data',
                                                    array=y_error,
                                                    visible=True)
                                                ))
                    if plot_par['ribbon'].values == True:
                        fig.add_trace(go.Scatter(x=x_data, y=y_data + y_error, mode='lines',
                                                name=plot_par['parameter_lab'][0], 
                                                line=dict(color=plot_par['colour'][0], width=0.5, dash = 'dot'),
                                                connectgaps=True,
                                                legendgroup=plot_par['parameter_lab'][0],
                                                showlegend=False,
                                                hoverinfo='skip'
                                                ))
                        fig.add_trace(go.Scatter(x=x_data, y=y_data - y_error, fill='tonexty', mode='lines',
                                                fillcolor=plot_par['fill'][0],
                                                name=plot_par['parameter_lab'][0], 
                                                line=dict(color=plot_par['colour'][0], width=0.5, dash = 'dot'),
                                                connectgaps=True,
                                                legendgroup=plot_par['parameter_lab'][0],
                                                showlegend=False,
                                                hoverinfo='skip'
                                                )) # fill to trace0 y

            #Modify plot
            fig.update_layout(
                margin=dict(l=100, r=220, b=15, t=15, pad=10),
                template="simple_white",
                font=dict(
                    family="Arial",
                    size=12,
                    color="black"
                ))
            fig.update_yaxes(title_text=fig_par['ylab'][chart], mirror=True)
            fig.update_xaxes(showgrid=True, showticklabels=False, ticks="",
                showline=True, mirror=True,
                range=[min(chart_dfs_mlt[chart].DateTime), max(chart_dfs_mlt[chart].DateTime)])
            
            #Special plot mods
            if fig_par['log'][chart] == True:
                fig.update_layout(yaxis_type="log")
                fig.update_yaxes(range=[math.log(fig_par['ymin'][chart], 10), math.log(fig_par['ymax'][chart], 10)])
            else:
                fig.update_yaxes(range=[fig_par['ymin'][chart], fig_par['ymax'][chart]])

            if plot == selected_plot_info['plot'].to_list()[len(selected_plot_info['plot'].to_list())-1]:
                fig.update_xaxes(showticklabels=True, ticks="outside")

            chart_figs[chart][plot] = fig
    else:
        chart_figs.append("")

#del chart, fig, i, plot, par, plot_par, x_data, y_data, y_error

Creating plots for chart 3: 100%|██████████| 12/12 [00:13<00:00,  1.08s/it]


In [120]:
chart_figs[0]["REXT"]

In [113]:
dcc_chart_figs = []
for chart in charts:
    if info['charts'].loc[chart, 'chart_status'] == 'ON':
        dcc_chart_figs.append([])
        p = 0
        for plot in chart_figs[chart]:
            if p != len(chart_figs[chart])-1:
                dcc_chart_figs[chart].append(
                    dcc.Graph(id='graph' + str(p),
                        figure=chart_figs[chart][plot],
                        style={'width': '98vw', 'height': '20vh'})
                    )
            else:
                dcc_chart_figs[chart].append(
                    dcc.Graph(id='graph' + str(p),
                        figure=chart_figs[chart][plot],
                        style={'width': '98vw', 'height': '25vh'})
                    )
            p = p + 1
    else:
        dcc_chart_figs.append("")

#del chart, p, plot

In [114]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
#external_stylesheets = [dbc.themes.LITERA]

app = dash.Dash(__name__, external_stylesheets = external_stylesheets)

tabs_init = []
for chart in charts:
    if info['charts'].loc[chart, 'chart_status'] == 'ON':
        tabs_init.append(dcc.Tab(label=info['charts']['chart_label'][chart], value="".join(["tab-", str(chart)])))

diff = datetime.now(timezone('UTC')) - date_end
last_date = humanize.naturaldelta(diff)
update_text = html.Div(html.P('Data last retrieved ' + last_date + ' ago'))

app.layout = html.Div(children=[
    html.Div([html.Img(src=app.get_asset_url('ToOL-PRO-BES_2.png'), style={'width':'90%', 'max-width': '100%'})], style={'textAlign': 'center'}),
    update_text,
    html.Div(children=[
        dcc.Tabs(id="tabs",
            value="tab-0",
            children=tabs_init),

        html.Div(children=[
            dcc.Loading(id='tabs-content')
        ]),
    ]),
])

@app.callback(Output('tabs-content', 'children'),
              [Input('tabs', 'value')])
def render_content(tab):
    time.sleep(2)
    if tab == 'tab-0':
        return html.Div(id='loading-0', children=dcc_chart_figs[0])
    elif tab == 'tab-1':
        return html.Div(id='loading-1', children=dcc_chart_figs[1])
    elif tab == 'tab-2':
        return html.Div(id='loading-2', children=dcc_chart_figs[2])
    elif tab == 'tab-3':
        return html.Div(id='loading-3', children=dcc_chart_figs[3])

In [115]:
if __name__ == '__main__':
    app.run_server()
    #debug=True, dev_tools_hot_reload_interval=5000)
                   #dev_tools_hot_reload_max_retry=30)

* Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [21/Feb/2020 13:45:02] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/Feb/2020 13:45:03] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [21/Feb/2020 13:45:03] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [21/Feb/2020 13:45:03] "GET /_favicon.ico?v=1.8.0 HTTP/1.1" 200 -
127.0.0.1 - - [21/Feb/2020 13:45:06] "POST /_dash-update-component HTTP/1.1" 200 -


In [ ]:
graphs = [
    'https://plot.ly/~christopherp/308',
    'https://plot.ly/~christopherp/306',
    'https://plot.ly/~christopherp/300',
    'https://plot.ly/~christopherp/296'
]

In [ ]:
from IPython.display import display, HTML

def report_block_template(report_type, graph_url, caption=''):
    if report_type == 'interactive':
        graph_block = '<iframe style="border: none;" src="{graph_url}.embed" width="100%" height="600px"></iframe>'
    elif report_type == 'static':
        graph_block = (''
            '<a href="{graph_url}" target="_blank">' # Open the interactive graph when you click on the image
                '<img style="height: 400px;" src="{graph_url}.png">'
            '</a>')

    report_block = ('' +
        graph_block +
        '{caption}' + # Optional caption to include below the graph
        '<br>'      + # Line break
        '<a href="{graph_url}" style="color: rgb(190,190,190); text-decoration: none; font-weight: 200;" target="_blank">'+
            'Click to comment and see the interactive graph' + # Direct readers to Plotly for commenting, interactive graph
        '</a>' +
        '<br>' +
        '<hr>') # horizontal line                       

    return report_block.format(graph_url=graph_url, caption=caption)


interactive_report = ''
static_report = ''

for graph_url in graphs:
    _static_block = report_block_template('static', graph_url, caption='')
    _interactive_block = report_block_template('interactive', graph_url, caption='')

    static_report += _static_block
    interactive_report += _interactive_block

In [ ]:
from xhtml2pdf import pisa             # import python module

# Utility function
def convert_html_to_pdf(source_html, output_filename):
    # open output file for writing (truncated binary)
    result_file = open(output_filename, "w+b")

    # convert HTML to PDF
    pisa_status = pisa.CreatePDF(
            source_html,                # the HTML to convert
            dest=result_file)           # file handle to recieve result

    # close output file
    result_file.close()                 # close output file

    # return True on success and False on errors
    return pisa_status.err

In [ ]:
convert_html_to_pdf(static_report, 'report.pdf')

In [ ]:
chart = 0
for plot in chart_figs[chart]:
    print(plot)

In [ ]:
import os, shutil

def delete_folder_contents(folder):
    for filename in os.listdir(folder):
        file_path = os.path.join(folder, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))

In [ ]:
if not os.path.exists("Temp_Data"):
    os.mkdir("Temp_Data")

for chart in tqdm(charts, desc="Exporting SVGs"):
    if info['charts'].loc[chart, 'chart_status'] == 'ON':
        folder = "Temp_Data/" + str(chart) + "_" + info['charts'].loc[chart, 'chart']
        if not os.path.exists(folder):
            os.mkdir(folder)
        else:
            delete_folder_contents(folder)
        p = 0
        for plot in chart_figs[chart]:
            chart_figs[chart][plot].write_image(folder + "/" + str(p) + "_" + plot + ".svg", width=210*6, height=29.7*6, scale=1)
            p = p + 1

In [ ]:
tab1_content = dbc.Card(
    dbc.CardBody(
        [
            html.P("This is tab 1!", className="card-text"),
            dbc.Button("Click here", color="success"),
        ]
    ),
    className="mt-3",
)

tab2_content = dbc.Card(
    dbc.CardBody(
        [
            html.P("This is tab 2!", className="card-text"),
            dbc.Button("Don't click here", color="danger"),
        ]
    ),
    className="mt-3",
)


tabs = dbc.Tabs(
    [
        dbc.Tab(tab1_content, label="Tab 1"),
        dbc.Tab(tab2_content, label="Tab 2"),
        dbc.Tab(
            "This tab's content is never seen", label="Tab 3", disabled=True
        ),
    ]
)

layout = html.Div([tabs])

app = dash.Dash(__name__, external_stylesheets=[dbc.themes.LITERA]

app.layout = layout